# Ejercicio AirBnB

In [ ]:
import pandas as pd
import openai
import os
from IPython.display import Markdown
import requests
import json
from dotenv import load_dotenv
import time
from requests.exceptions import RequestException

load_dotenv()
auth_token = os.getenv("AUTH_TOKEN")
headers = {
  'Content-Type': 'application/json',
  'Authorization': f"Bearer {auth_token}"
}
url = 'https://api.awanllm.com/v1/chat/completions'  # URL for the API

In [ ]:

def preguntar_llama(sys_prompt, prompt, max_retries=3, initial_delay=10, timeout=60, rate_limit_delay=5):
    """
    Sends a prompt to an AI model and retrieves the response.

    Args:
        sys_prompt (str): The system prompt to be sent to the AI model.
        prompt (str): The user prompt to be sent to the AI model.
        max_retries (int, optional): The maximum number of retries in case of errors. Defaults to 3.
        initial_delay (int, optional): The initial delay between retries in seconds. Defaults to 1.
        timeout (int, optional): The timeout for the HTTP request in seconds. Defaults to 60.
        rate_limit_delay (int, optional): The delay between retries in case of rate limiting errors. Defaults to 5.

    Returns:
        str: The response generated by the AI model.

    Raises:
        requests.exceptions.HTTPError: If an HTTP error occurs during the request.
        requests.exceptions.Timeout: If the request times out.
        Exception: If an unexpected error occurs.

    """
    payload = json.dumps({
        "model": "Meta-Llama-3-8B-Instruct",
        "messages": [
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": prompt}
        ],
    })
    
    for attempt in range(max_retries):
        try:
            response = requests.post(url, headers=headers, data=payload, timeout=timeout)
            response.raise_for_status()
            json_response = response.json()
            return json_response['choices'][0]['message']['content']
        except requests.exceptions.HTTPError as e:
            status_code = e.response.status_code
            if status_code == 429:
                print(f"Intento {attempt + 1} fallido. Error: {e}. Demasiadas solicitudes.")
                delay = rate_limit_delay
            elif status_code == 502:
                print(f"Intento {attempt + 1} fallido. Error: {e}. Error de Gateway.")
                delay = initial_delay
            else:
                print(f"Intento {attempt + 1} fallido. Error: {e}")
                break  # Salir del bucle si hay un error HTTP que no sea 429 o 502
            
            print(f"Reintentando en {delay} segundos...")
            time.sleep(delay)
        except requests.exceptions.Timeout as e:
            print(f"Intento {attempt + 1} fallido por tiempo de espera excedido: {e}")
            if attempt < max_retries - 1:
                print("Reintentando...")
            time.sleep(initial_delay * (2 ** attempt))
        except Exception as e:
            print(f"Error no esperado: {e}")
            if attempt == max_retries - 1:
                return None  # Retornar None solo después de todos los intentos

    print("No se pudo completar la solicitud después de varios intentos.")
    return None

data = pd.read_csv('listings1_cleaned.csv')
data = data[['listing_id', 'name', 'description']]
data['text'] = data['name'].fillna('') + '\n' + data['description'].fillna('')
data = data.sample(250)

llama_responses = {}

prompt_template = '''
Vas a analizar los comentarios de unos alojamientos turísticos. 
Se te va a proporcionar una serie de textos que has de analizar y clasificar en un json con la siguiente estructura:
```json
{
    "sentiment": str,
    "language": str,
    "3GoodThings": list,
    "3BadThings": list,
}
```
Donde:
- sentiment es la polaridad del texto (positivo, negativo o neutro)
- language es el idioma del texto (es, en, fr, de, it, pt)
- 3GoodThings es una lista con las 3 cosas buenas del texto
- 3BadThings es una lista con las 3 cosas malas del texto
Analiza el siguiente texto y clasifícalo en el json correspondiente: 
```
{{text}}
```
'''
for index, row in data.iterrows():
    text = str(row['text'])
    listing_id = row['listing_id']
    
    prompt = prompt_template.replace('{{text}}', text)
    
    llama_response = preguntar_llama('Eres un asistente de IA cuyo trabajo es responder a lo que te está pidiendo el usuario de la forma más fiel al prompt sin preámbulos explicativos ni resúmenes finales.', prompt)
    if llama_response is not None:
        llama_responses[listing_id] = llama_response
    else:
        print(f"No se pudo obtener una respuesta para el listing_id: {listing_id}")

data['llama_response'] = data['listing_id'].map(llama_responses)
data

In [ ]:
data.to_csv('Airbnb_llama_responses_250.csv', index=False)

In [ ]:
data = pd.read_csv('Airbnb_llama_responses_250.csv')

In [ ]:
def process_json_response(json_str):
    # Verificar si json_str es NaN
    if json_str != json_str:  # Esta es una forma de verificar NaN en Python
        # Devolver un diccionario vacío o con un mensaje de error
        return {"error": "Input is NaN, cannot process."}
    
    try:
        # Intentar cargar el JSON
        json_data = json.loads(json_str)
        # Si es exitoso, devolver el diccionario
        return json_data
    except json.JSONDecodeError:
        # Si falla, devolver un diccionario vacío
        return {}
# Aplicar la función a la columna 'llama_response'
data['processed_response'] = data['llama_response'].apply(process_json_response)

# Obtener todas las claves únicas de los JSONs
all_keys = set()
for response in data['processed_response']:
    all_keys.update(response.keys())

# Crear nuevas columnas para cada clave
for key in all_keys:
    data[key] = data['processed_response'].apply(lambda x: x.get(key, None))

# Eliminar la columna temporal 'processed_response'
data = data.drop('processed_response', axis=1)

data

In [ ]:
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
import ast

# Función para convertir string de lista a lista real
def string_to_list(x):
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)
        except:
            return x
    return x

# Función para contar elementos en listas
def count_list_elements(series):
    counter = {}
    for item in series.dropna():
        if isinstance(item, list):
            for element in item:
                counter[element] = counter.get(element, 0) + 1
        else:
            counter[item] = counter.get(item, 0) + 1
    return pd.Series(counter)

# Función para crear gráfica según el tipo de dato
def create_plot(df, column):
    series = df[column].apply(string_to_list)
    if series.empty:
        fig = go.Figure()
        fig.add_annotation(text=f"No hay datos para la columna {column}", xref="paper", yref="paper", showarrow=False)
    elif series.dtype == 'object':
        # Para datos categóricos (string) o listas
        value_counts = count_list_elements(series)
        if not value_counts.empty:
            fig = px.bar(x=value_counts.index, y=value_counts.values, title=f'Distribución de {column}')
        else:
            fig = go.Figure()
            fig.add_annotation(text=f"No hay datos no nulos para la columna {column}", xref="paper", yref="paper", showarrow=False)
    elif series.dtype in ['int64', 'float64']:
        # Para datos numéricos
        if series.notna().any():
            fig = px.histogram(df, x=column, title=f'Distribución de {column}')
        else:
            fig = go.Figure()
            fig.add_annotation(text=f"No hay datos numéricos no nulos para la columna {column}", xref="paper", yref="paper", showarrow=False)
    elif series.dtype == 'bool':
        # Para datos booleanos
        value_counts = series.value_counts()
        if not value_counts.empty:
            fig = px.pie(values=value_counts.values, names=value_counts.index, title=f'Distribución de {column}')
        else:
            fig = go.Figure()
            fig.add_annotation(text=f"No hay datos booleanos no nulos para la columna {column}", xref="paper", yref="paper", showarrow=False)
    else:
        # Para otros tipos de datos, mostrar un mensaje
        fig = go.Figure()
        fig.add_annotation(text=f"No se puede graficar la columna {column} (tipo: {series.dtype})", xref="paper", yref="paper", showarrow=False)
    
    fig.update_layout(title_x=0.5)  # Centrar el título
    return fig

# Obtener las nuevas columnas (excluyendo las columnas originales)
new_columns = [col for col in data.columns if col not in ['listing_id', 'name', 'description', 'text', 'llama_response']]

# Imprimir información sobre las nuevas columnas
for column in new_columns:
    print(f"Columna: {column}")
    print(f"Tipo de datos: {data[column].dtype}")
    print(f"Número de valores no nulos: {data[column].count()}")
    print(f"Ejemplo de valor: {data[column].iloc[0]}")
    print("---")

# Crear subplots
fig = make_subplots(rows=len(new_columns), cols=1, subplot_titles=new_columns)

# Crear una gráfica para cada nueva columna
for i, column in enumerate(new_columns, start=1):
    subplot = create_plot(data, column)
    for trace in subplot.data:
        fig.add_trace(trace, row=i, col=1)

# Actualizar el diseño
fig.update_layout(height=500*len(new_columns), title_text="Visualizaciones de las nuevas columnas", showlegend=False)

# Mostrar la figura
fig.show()